In [ ]:
# Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.
# Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).
# 1. С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
# 2. Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, 
# чтобы в качестве строк были категории продуктов,
# в качестве столбцов были дни, 
# а значениями были средние значения выручки от продаж за соответствующий день

In [1]:
import pyspark

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month
from pyspark.sql.functions import countDistinct, col, collect_list, rank, row_number, first, desc, round
from pyspark.sql.window import Window
import pandas as pd

In [3]:
spark = SparkSession.builder.appName('HomeWork#3').getOrCreate()

24/04/21 09:25:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 09:25:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [15]:
data = {("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)}
columns = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data=data, schema=columns)
df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-21|      Books|     200|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-20|      Books|     150|   9000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|Electronics|     100|  12000|
|2023-11-22|      Books|     180|  10000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-20|   Clothing|     300|  15000|
+----------+-----------+--------+-------+



In [16]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- revenue: long (nullable = true)



In [17]:
df = df.withColumn('quantity', col('quantity').cast('int'))
df = df.withColumn('revenue', col('revenue').cast('int'))
df = df.withColumn('date', col('date').cast('date'))
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- revenue: integer (nullable = true)



In [9]:
# 1. С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

In [26]:
df.withColumn('revenue_avg_by_category', round(avg('revenue').over(Window.partitionBy('category')), 2)).show()

+----------+-----------+--------+-------+-----------------------+
|      date|   category|quantity|revenue|revenue_avg_by_category|
+----------+-----------+--------+-------+-----------------------+
|2023-11-21|Electronics|     110|  13000|                12500.0|
|2023-11-20|Electronics|     100|  12000|                12500.0|
|2023-11-22|Electronics|     105|  12500|                12500.0|
|2023-11-22|   Clothing|     320|  16000|                15000.0|
|2023-11-21|   Clothing|     280|  14000|                15000.0|
|2023-11-20|   Clothing|     300|  15000|                15000.0|
|2023-11-21|      Books|     200|  12000|               10333.33|
|2023-11-20|      Books|     150|   9000|               10333.33|
|2023-11-22|      Books|     180|  10000|               10333.33|
+----------+-----------+--------+-------+-----------------------+



In [27]:
# 2. Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, 
# чтобы в качестве строк были категории продуктов,
# в качестве столбцов были дни, 
# а значениями были средние значения выручки от продаж за соответствующий день

In [32]:
df.groupBy('category').pivot('date').agg(round(avg('revenue'),2)).show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

